# Adaptive Sampling

In this tutorial you will learn:
    
    ...
    
    

In [ ]:
from bluesky import RunEngine
from bluesky.plans import count
from utils.simulated_hardware import detector, sample_selector, select_sample
from utils.visualization import SummedImages, JupyterFigure
from bluesky_widgets.utils.streaming import stream_documents_into_runs
from bluesky_widgets.models.plot_specs import Axes, Figure

In [ ]:
axes_list = [Axes() for _ in range(9)]
figure_model = Figure(axes_list, title="Summed Exposures")
models = [SummedImages("detector_image", axes=axes) for axes in axes_list]
figure = JupyterFigure(figure_model)
figure

In [ ]:
RE = RunEngine()

In [ ]:
RE(select_sample(0))

In [ ]:
RE(count([detector], 10, delay=1), stream_documents_into_runs(models[0].add_run))